In [1]:
import pandas as pd

In [30]:
# save data as variables
uber = pd.read_csv('uber-raw-data-june-15.csv')
zone = pd.read_csv('taxi_zone_lookup.csv')

In [31]:
# check data quickly
uber

,Unnamed: 0,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,1,B02764,2015-06-01 10:25:00,B02682,107
1,2,B02764,2015-06-01 10:25:00,B02764,170
2,3,B02764,2015-06-01 10:25:00,B02764,237
3,4,B02764,2015-06-01 10:25:00,B02764,229
4,5,B02764,2015-06-01 10:25:00,B00013,50
...,...,...,...,...,...
2816890,2816891,B02836,2015-06-30 22:51:00,B02836,143
2816891,2816892,B02836,2015-06-30 22:52:00,B02836,90
2816892,2816893,B02836,2015-06-30 23:06:00,B02836,246
2816893,2816894,B02836,2015-06-30 23:20:00,B02836,107


In [32]:
# re-name zone data columns
zone.columns = ['locationID', 'borough', 'zone', 'service_zone']

In [33]:
zone.head()

,locationID,borough,zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [34]:
# group uber data by locationID and produce count
uber.groupby(by='locationID').count()

,Unnamed: 0,Dispatching_base_num,Pickup_date,Affiliated_base_num
locationID,,,,
1,29,29,29,29
2,1,1,1,1
3,860,860,860,860
4,17608,17608,17608,17608
5,10,10,10,10
...,...,...,...,...
260,4481,4481,4481,4481
261,14239,14239,14239,14239
262,20909,20909,20909,20909


In [35]:
# merge uber data with zone data on locationID
uber1 = uber.merge(zone, how='outer', on='locationID')

In [36]:
uber1

,Unnamed: 0,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,borough,zone,service_zone
0,1.0,B02764,2015-06-01 10:25:00,B02682,107,Manhattan,Gramercy,Yellow Zone
1,14.0,B02764,2015-06-01 10:26:00,B02764,107,Manhattan,Gramercy,Yellow Zone
2,54.0,B02764,2015-06-01 10:28:00,B00035,107,Manhattan,Gramercy,Yellow Zone
3,83.0,B02764,2015-06-01 10:29:00,B02253,107,Manhattan,Gramercy,Yellow Zone
4,108.0,B02764,2015-06-01 10:30:00,B02764,107,Manhattan,Gramercy,Yellow Zone
...,...,...,...,...,...,...,...,...
2816896,NaN,NaN,NaN,NaN,103,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone
2816897,NaN,NaN,NaN,NaN,104,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone
2816898,NaN,NaN,NaN,NaN,110,Staten Island,Great Kills Park,Boro Zone
2816899,NaN,NaN,NaN,NaN,199,Bronx,Rikers Island,Boro Zone


In [37]:
# drop all na values in merged data
uber1.dropna(inplace=True)

In [44]:
# group data by zone as a count
uber2 = uber1.groupby(by='zone', as_index=False).count()

In [57]:
# some zone names are conjoined together in uber data with a '/' while these places are considered separate neighborhoods in our nyc_geo data
# first split these places that are conjoined by '/'
uber2['zone'] = uber2['zone'].str.split('/')

In [60]:
# use explode to separate the split places into different rows as their own neighborhood to match nyc_geo data
uber3 = uber2.explode('zone')

In [63]:
# drop all columns except two
uber4 = uber3.drop(columns=['Dispatching_base_num', 'Pickup_date', 'Affiliated_base_num', 'locationID', 'borough', 'service_zone'])

In [90]:
# rename remaining columns to name (to match nyc_geo data) and uber_count (the data we want to join onto nyc_geo data)
uber4.columns = ['name', 'uber_count']

In [91]:
uber4.reset_index(drop=True, inplace=True)

In [93]:
# group uber data by name as a sum of their counts now
uber5 = uber4.groupby(by='name', as_index=False).sum()

In [94]:
# there is a large discrepancy between uber data names and nyc_geo names
# many of the same neighborhoods have minor differences that is making merge impossible without losing too much data (almost 50% data loss)
# another option was to artificially inflate the number of neighborhoods we had with all of these slightly different names
# the chosen method was to preserve the number of original sample points for compatibility with future data joins

# function to fully print entire dataframe for visual
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [224]:
print_full(uber5)

                             name  uber_count
0                        Allerton         860
1                   Alphabet City       17608
2                        Annadale          11
3                   Arden Heights          10
4                        Arrochar         145
5                         Arverne         241
6                         Astoria       18384
7                 Astoria Heights          97
8                      Auburndale         647
9                    Baisley Park        1697
10                     Bath Beach         709
11                     Bath Beach         920
12              Battery Park City         710
13              Battery Park City       29335
14                      Bay Ridge        7586
15                    Bay Terrace         685
16                        Bayside        1964
17                   Bedford Park       17455
18                   Bedford Park        1449
19                      Bellerose         713
20                        Belmont 

In [223]:
# manually update all discrepancies by manually cross-referencing between uber data names and nyc_geo data names
uber5.at[308, 'name'] = "Yorkville"

In [85]:
# read nyc_geo data
nyc_geo = pd.read_csv('nyc_geo_dataframe.csv')

In [87]:
# sorting for clarity
nyc_geo1 = nyc_geo.sort_values(by='name')

In [225]:
# noticed that there were duplicate names in the nyc_geo data (they had slightly different coordinates but were largely the same place)
nyc_geo2 = nyc_geo1.drop_duplicates(subset='name', keep='first')

In [227]:
uber6 = uber5.groupby(by='name').sum()

In [229]:
# merge completed uber data and nyc_geo data
nyc_uber = nyc_geo2.merge(uber6, how='left', on='name')

In [232]:
# fill all na values in merged data with 0 (this may be changed later as needed)
nyc_uber.fillna(value=0, inplace=True)

In [233]:
nyc_uber

,name,longitude,latitude,uber_count
0,Allerton,-73.859319,40.865788,860.0
1,Annadale,-74.178549,40.538114,11.0
2,Arden Heights,-74.185887,40.549286,10.0
3,Arlington,-74.165104,40.635325,0.0
4,Arrochar,-74.067124,40.596313,145.0
...,...,...,...,...
297,Woodhaven,-73.858110,40.689887,2058.0
298,Woodlawn,-73.867315,40.898273,810.0
299,Woodrow,-74.205246,40.541968,5.0
300,Woodside,-73.901842,40.746349,4567.0


In [234]:
# save merge data as csv
nyc_uber.to_csv('nyc_uber_data.csv', index=False)